In [12]:
from osgeo import gdal, gdalconst, ogr
from skimage import img_as_ubyte
import numpy as np
from IPython.display import Image, display

from Utils.getPoints import pointsAsPixels

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as TF 
import random

In [13]:
print(torch.__version__)

2.0.1


In [3]:
shapefile_list = ["06D2/S1A_EW_GRDM_1SDH_20230212T135259_20230212T135341_047203_05A9F4_06D2.shp",
                  "17F5/S1A_EW_GRDM_1SDH_20230212T135159_20230212T135259_047203_05A9F4_17F5.shp",
                  "84EF/S1A_EW_GRDM_1SDH_20230118T131123_20230118T131232_046838_059DBE_84EF.shp",
                  "264D/S1A_EW_GRDM_1SDH_20230209T150434_20230209T150539_047160_05A886_264D.shp",
                  "D15B/S1A_EW_GRDM_1SDH_20230118T131019_20230118T131123_046838_059DBE_D15B.shp",
                  "E82B/S1A_EW_GRDM_1SDH_20230209T150539_20230209T150639_047160_05A886_E82B.shp"]

sar_image_list = ["SNAP_Images/06D2/S1A_EW_GRDM_1SDH_20230212T135259_20230212T135341_047203_05A9F4_06D2_Orb_NR_Cal_Spk_THR_SHP_EC.tif",
            "SNAP_Images/17F5/S1A_EW_GRDM_1SDH_20230212T135159_20230212T135259_047203_05A9F4_17F5_Orb_NR_Cal_Spk_THR_SHP_EC.tif",
            "SNAP_Images/84EF/S1A_EW_GRDM_1SDH_20230118T131123_20230118T131232_046838_059DBE_84EF_Orb_NR_Cal_Spk_THR_SHP_EC.tif",
            "SNAP_Images/264D/S1A_EW_GRDM_1SDH_20230209T150434_20230209T150539_047160_05A886_264D_Orb_NR_Cal_Spk_THR_SHP_EC.tif",
            "SNAP_Images/D15B/S1A_EW_GRDM_1SDH_20230118T131019_20230118T131123_046838_059DBE_D15B_Orb_NR_Cal_Spk_THR_SHP_EC.tif",
            "SNAP_Images/E82B/S1A_EW_GRDM_1SDH_20230209T150539_20230209T150639_047160_05A886_E82B_Orb_NR_Cal_Spk_THR_SHP_EC.tif",]


Simple CNN

In [4]:
# CNN Model
class CNN(nn.Module):
    def __init__(self, height, width):
        super(CNN, self).__init__()
        self.height = height
        self.width = width
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        fc_layers = nn.Sequential(
            nn.Linear(x.shape[1], 1),
            nn.Sigmoid()
        )
        x = fc_layers(x)
        return x


In [5]:
batch_size = 64
shape = 30
model = CNN(height = shape, width = shape)

criterion = nn.BCEWithLogitsLoss() # nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) 

In [6]:

for index in range(len(sar_image_list)):
    # import shapefiles and image
    # Open the SAR data and get transform
    sar_image = sar_image_list[index]
    shapefile = shapefile_list[index]
    print(sar_image)
    print(shapefile)

    ds = gdal.Open(sar_image, gdalconst.GA_ReadOnly)
    image = np.asarray(ds.GetRasterBand(1).ReadAsArray())

    image = img_as_ubyte(image)
    height, width = image.shape

    gt=ds.GetGeoTransform()
    px, py = pointsAsPixels(shapefile, gt)

    label_list = np.zeros_like(image)
    for (x, y) in zip(px, py):
        label_list[y][x] = 1

    xsize, ysize = image.shape

    img_list = []

    for (x, y) in zip(px, py):
        x_max = min(x + shape//2, xsize)
        x_min = max(x - shape//2, 0)
        y_max = min(y + shape//2, ysize)
        y_min = max(y - shape//2, 0)
        sub = image[y_min:y_max, x_min:x_max]
        img_list.append((sub, 1))

    newxs = [random.randint(15, xsize-15) for _ in range(len(img_list)//2)]
    newys = [random.randint(15, ysize-15) for _ in range(len(img_list)//2)]
    for (x, y) in zip(newxs, newys):
        while x in px and y in px:
            x = random.randint(0, xsize)
            y = random.randint(0, ysize)
        x_max = x + shape//2
        x_min = x - shape//2
        y_max = y + shape//2
        y_min = y - shape//2
        sub = image[y_min:y_max, x_min:x_max]
        if sub.shape == (30, 30):
            img_list.append((sub, 0))
            
    print("training")
    train_loader = DataLoader(img_list, batch_size=batch_size, shuffle=True)
    num_epochs = 50
    for epoch in range(num_epochs):
        epoch_loss = 0
        for images, l in train_loader:
            images = images.float()
            optimizer.zero_grad()
            images = images.unsqueeze(1)
            l = l.unsqueeze(1).float()
            outputs = model(images)
            loss = criterion(outputs, l)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        epoch_loss /= len(train_loader)
        if epoch % 5 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss : {epoch_loss}")

    print()
        

SNAP_Images/06D2/S1A_EW_GRDM_1SDH_20230212T135259_20230212T135341_047203_05A9F4_06D2_Orb_NR_Cal_Spk_THR_SHP_EC.tif
project/06D2/06D2.shp


Warning 1: SNAP_Images/06D2/S1A_EW_GRDM_1SDH_20230212T135259_20230212T135341_047203_05A9F4_06D2_Orb_NR_Cal_Spk_THR_SHP_EC.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
ERROR 1: PROJ: proj_identify: SQLite error on SELECT name, coordinate_system_auth_name, coordinate_system_code, geodetic_crs_auth_name, geodetic_crs_code, conversion_auth_name, conversion_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM projected_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name


training
Epoch [1/50], Average Loss : 0.5592577457427979
Epoch [6/50], Average Loss : 0.652182400226593
Epoch [11/50], Average Loss : 0.600923478603363
Epoch [16/50], Average Loss : 0.6342452764511108
Epoch [21/50], Average Loss : 0.5244070887565613
Epoch [26/50], Average Loss : 0.6687339544296265
Epoch [31/50], Average Loss : 0.5461313128471375
Epoch [36/50], Average Loss : 0.584213376045227
Epoch [41/50], Average Loss : 0.6583733558654785
Epoch [46/50], Average Loss : 0.6510501503944397

SNAP_Images/84EF/S1A_EW_GRDM_1SDH_20230118T131123_20230118T131232_046838_059DBE_84EF_Orb_NR_Cal_Spk_THR_SHP_EC.tif
project/84EF/84EF.shp


Warning 1: SNAP_Images/84EF/S1A_EW_GRDM_1SDH_20230118T131123_20230118T131232_046838_059DBE_84EF_Orb_NR_Cal_Spk_THR_SHP_EC.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
ERROR 1: PROJ: proj_identify: SQLite error on SELECT name, coordinate_system_auth_name, coordinate_system_code, geodetic_crs_auth_name, geodetic_crs_code, conversion_auth_name, conversion_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM projected_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name


training
Epoch [1/50], Average Loss : 0.6376059293746948
Epoch [6/50], Average Loss : 0.6427482277154922
Epoch [11/50], Average Loss : 0.6370945066213608
Epoch [16/50], Average Loss : 0.6365340620279312
Epoch [21/50], Average Loss : 0.6381855100393296
Epoch [26/50], Average Loss : 0.6381023705005646
Epoch [31/50], Average Loss : 0.6377218782901763
Epoch [36/50], Average Loss : 0.6369765758514404
Epoch [41/50], Average Loss : 0.637134113907814
Epoch [46/50], Average Loss : 0.6371440291404724

SNAP_Images/264D/S1A_EW_GRDM_1SDH_20230209T150434_20230209T150539_047160_05A886_264D_Orb_NR_Cal_Spk_THR_SHP_EC.tif
project/264D/264D.shp


...

training
Epoch [1/50], Average Loss : 0.6469286719718611
Epoch [6/50], Average Loss : 0.6389012968683817
Epoch [11/50], Average Loss : 0.6404496539069946
Epoch [16/50], Average Loss : 0.6389202568904463
Epoch [21/50], Average Loss : 0.640521810715457
Epoch [26/50], Average Loss : 0.6384668027062014
Epoch [31/50], Average Loss : 0.6393797354525831
Epoch [36/50], Average Loss : 0.640492371765964
Epoch [41/50], Average Loss : 0.6395601759473961
Epoch [46/50], Average Loss : 0.6404328999749149

SNAP_Images/D15B/S1A_EW_GRDM_1SDH_20230118T131019_20230118T131123_046838_059DBE_D15B_Orb_NR_Cal_Spk_THR_SHP_EC.tif
project/D15B/D15B.shp


Warning 1: SNAP_Images/D15B/S1A_EW_GRDM_1SDH_20230118T131019_20230118T131123_046838_059DBE_D15B_Orb_NR_Cal_Spk_THR_SHP_EC.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
ERROR 1: PROJ: proj_identify: SQLite error on SELECT name, coordinate_system_auth_name, coordinate_system_code, geodetic_crs_auth_name, geodetic_crs_code, conversion_auth_name, conversion_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM projected_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name


training
Epoch [1/50], Average Loss : 0.638410480862314
Epoch [6/50], Average Loss : 0.6384465382857756
Epoch [11/50], Average Loss : 0.6382329430092465
Epoch [16/50], Average Loss : 0.6382271166552197
Epoch [21/50], Average Loss : 0.6387201114134355
Epoch [26/50], Average Loss : 0.6382985264062881
Epoch [31/50], Average Loss : 0.6390989402478392
Epoch [36/50], Average Loss : 0.6389329203150489
Epoch [41/50], Average Loss : 0.6392116228287871
Epoch [46/50], Average Loss : 0.6387069929729808

SNAP_Images/E82B/S1A_EW_GRDM_1SDH_20230209T150539_20230209T150639_047160_05A886_E82B_Orb_NR_Cal_Spk_THR_SHP_EC.tif
project/E82B/E82B.shp


Warning 1: SNAP_Images/E82B/S1A_EW_GRDM_1SDH_20230209T150539_20230209T150639_047160_05A886_E82B_Orb_NR_Cal_Spk_THR_SHP_EC.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
ERROR 1: PROJ: proj_identify: SQLite error on SELECT name, coordinate_system_auth_name, coordinate_system_code, geodetic_crs_auth_name, geodetic_crs_code, conversion_auth_name, conversion_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM projected_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name


training
Epoch [1/50], Average Loss : 0.6394722064336141
Epoch [6/50], Average Loss : 0.6346553762753805
Epoch [11/50], Average Loss : 0.638267437616984
Epoch [16/50], Average Loss : 0.6421972711881002
Epoch [21/50], Average Loss : 0.6425142685572306
Epoch [26/50], Average Loss : 0.6382994850476583
Epoch [31/50], Average Loss : 0.6381306250890096
Epoch [36/50], Average Loss : 0.6325142979621887
Epoch [41/50], Average Loss : 0.6403449773788452
Epoch [46/50], Average Loss : 0.6390873392422994



In [7]:
# import matplotlib.pyplot as plt
# fig = plt.figure()
# ax1 = fig.add_subplot(111)
# ax1.imshow(image)
# ax1.contour(label_list)

In [11]:
# Specify a path
PATH = "CNN_model.pt"
# Save
torch.save(model, PATH)
print("Saving Model state to", PATH)

Saving Model state to CNN_model.pt
